In [ ]:
# Run this cell once in Colab to install all necessary libraries
# Install required packages for embedding, GPT, similarity search, and model loading
!pip install transformers sentence-transformers faiss-cpu openai huggingface_hub gdown --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.6 MB/s eta 0:00:00


In [ ]:
import os
from zipfile import ZipFile
from PIL import Image
import torch
import numpy as np
import faiss
import json
from transformers import BlipProcessor, BlipForConditionalGeneration
from sentence_transformers import SentenceTransformer
from google.colab import drive, files, userdata
from huggingface_hub import login
import shutil
import gdown
import requests
from difflib import SequenceMatcher
import openai

In [ ]:
# Construct direct download URL
url = "https://drive.google.com/uc?id=YOUR_GOOGLE_DRIVE_FILE_ID"
output = "dataset.zip"

# Download the file
gdown.download(url, output, quiet=False)

# Unzip into target folder
with ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall("unzipped_images")

Downloading...
From (original): https://drive.google.com/uc?id=1_RW0O8w8aXqXD72K5hyNeI30n6GUSGFW
From (redirected): https://drive.google.com/uc?id=1_RW0O8w8aXqXD72K5hyNeI30n6GUSGFW&confirm=t&uuid=6cc57c23-fd92-4fdb-8907-12a926ed02e6
To: /content/ElvtrHW3_480p.zip
100%|██████████| 300M/300M [00:02<00:00, 146MB/s]


In [ ]:
# === CONFIG ===
FORCE_DEVICE = "auto"  # Options: "cpu", "cuda", or "auto"
BATCH_SIZE = 8
CAPTION_PATH = "captions_partial.json"
ZIP_FILE = next(iter(uploaded))  # Use the uploaded filename dynamically

In [ ]:
# === DEVICE SETUP ===
device = "cuda" if torch.cuda.is_available() and FORCE_DEVICE != "cpu" else "cpu"
print(f"🚀 Using device: {device.upper()}")

🚀 Using device: CPU


In [ ]:
# === FIND IMAGE FOLDER ===
root = "unzipped_images"
subfolders = [f.path for f in os.scandir(root) if f.is_dir()]
if not subfolders:
    raise ValueError("No folders found inside unzipped folder.")
image_folder = subfolders[0]

In [ ]:
# === GATHER IMAGES ===
image_files = []
for r, _, fns in os.walk(image_folder):
    for fn in fns:
        if fn.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_files.append(os.path.join(r, fn))

print(f"🖼️ Found {len(image_files)} images")

🖼️ Found 3719 images


In [ ]:
# === PROMPT FOR HUGGING FACE TOKEN  ===
# Make sure Notebook Access is enabled
# Then, add your Hugging Face token using 'HF_TOKEN' as the key
hf_token = userdata.get('HF_TOKEN')

# Login using the secret
login(token=hf_token)

In [ ]:
# === LOAD BLIP MODEL ===
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base", use_fast=False)
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
# === LOAD/RESUME CAPTIONS ===
if os.path.exists(CAPTION_PATH):
    with open(CAPTION_PATH, "r") as f:
        captions = json.load(f)
    print(f"🔁 Resuming from {len(captions)} captions")
else:
    captions = {}

In [ ]:
# === BATCH CAPTIONING ===
def batch_caption(paths):
    images = [Image.open(p).convert("RGB") for p in paths]
    inputs = processor(images=images, return_tensors="pt", padding=True).to(device)
    outputs = blip_model.generate(**inputs)
    return [processor.decode(o, skip_special_tokens=True) for o in outputs]

pending = [p for p in image_files if p not in captions]
total = len(pending)

for i in range(0, total, BATCH_SIZE):
    batch_paths = pending[i:i+BATCH_SIZE]
    try:
        batch_captions = batch_caption(batch_paths)
        for path, caption in zip(batch_paths, batch_captions):
            captions[path] = caption
    except Exception as e:
        print(f"⚠️ Failed batch at {i}: {e}")
        continue

    if i % (BATCH_SIZE * 2) == 0:
        print(f"📸 Captioned {i}/{total}")
        with open(CAPTION_PATH, "w") as f:
            json.dump(captions, f)

with open("captions_final.json", "w") as f:
    json.dump(captions, f)

print(f"✅ Done captioning {len(captions)} images")


📸 Captioned 0/3719
📸 Captioned 16/3719
📸 Captioned 32/3719
📸 Captioned 48/3719
📸 Captioned 64/3719
📸 Captioned 80/3719
📸 Captioned 96/3719
📸 Captioned 112/3719
📸 Captioned 128/3719
📸 Captioned 144/3719
📸 Captioned 160/3719
📸 Captioned 176/3719
📸 Captioned 192/3719
📸 Captioned 208/3719
📸 Captioned 224/3719
📸 Captioned 240/3719
📸 Captioned 256/3719
📸 Captioned 272/3719
📸 Captioned 288/3719
📸 Captioned 304/3719
📸 Captioned 320/3719
📸 Captioned 336/3719
📸 Captioned 352/3719
📸 Captioned 368/3719
📸 Captioned 384/3719
📸 Captioned 400/3719
📸 Captioned 416/3719
📸 Captioned 432/3719
📸 Captioned 448/3719
📸 Captioned 464/3719
📸 Captioned 480/3719
📸 Captioned 496/3719
📸 Captioned 512/3719
📸 Captioned 528/3719
📸 Captioned 544/3719
📸 Captioned 560/3719
📸 Captioned 576/3719
📸 Captioned 592/3719
📸 Captioned 608/3719
📸 Captioned 624/3719
📸 Captioned 640/3719
📸 Captioned 656/3719
📸 Captioned 672/3719
📸 Captioned 688/3719
📸 Captioned 704/3719
📸 Captioned 720/3719
📸 Captioned 736/3719
📸 Captioned 752/3719


In [ ]:
# === EMBED CAPTIONS ===
texts = list(captions.values())
try:
    embedder = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = embedder.encode(texts)
except:
    embeddings = np.array([[len(t)] * 5 for t in texts])

# === FAISS INDEX ===
faiss_index = faiss.IndexFlatL2(embeddings[0].shape[0])
faiss_index.add(np.array(embeddings))
print("📦 FAISS index built")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

📦 FAISS index built


In [ ]:
# === RETRIEVE MATCHES ===
def retrieve_top_matches(query, k=5):
    q_embed = embedder.encode([query])
    _, idxs = faiss_index.search(np.array(q_embed), k)
    items = list(captions.items())
    return [(os.path.basename(items[i][0]), texts[i]) for i in idxs[0]]

In [ ]:
# === SAMPLE QUERY 1 (Non-Agentic)===
query = "Are any cars blocking the bus lane?"
matches = retrieve_top_matches(query)

print(f"\n🔍 Top matches for: '{query}'")
for fname, cap in matches:
    print(f"• {fname}: {cap}")


🔍 Top matches for: 'Are any cars blocking the bus lane?'
• video1_frame_278.jpg: a road with cars and a bus on it
• video1_frame_74.jpg: a street with cars and a bus on it
• video1_frame_736.jpg: a street with cars and a bus on it
• video1_frame_282.jpg: a street with cars and a bus on it
• video1_frame_221.jpg: a street with cars and a bus on it


In [ ]:
# === SAMPLE QUERY 2 (Non-Agentic) ===
query = "Are any cars that are double parked?"
matches = retrieve_top_matches(query)

print(f"\n🔍 Top matches for: '{query}'")
for fname, cap in matches:
    print(f"• {fname}: {cap}")


🔍 Top matches for: 'Are any cars that are double parked?'
• video4_frame_34.jpg: two cars parked in a parking lot
• video4_frame_46.jpg: two cars parked in a parking lot
• video4_frame_50.jpg: two cars parked in a parking lot
• video4_frame_33.jpg: two cars parked in a parking lot
• video4_frame_310.jpg: two cars parked on the side of a road


In [ ]:
files.download("captions_final.json")